Print the version to check that Pandas is available.
If the module is not found, install pandas with `pip install pandas`.

In [2]:
import pandas as pd
print(pd.__version__)

1.2.4


Import the Json file with read_json. This will make a Pandas [*DataFrame*](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (a table).

In [3]:
df = pd.read_json("Application_Log.json")


Print the first three rows to see what the table looks like.

In [6]:
df.head(5)

,ticks,type,event,arg1,arg2,arg3,arg4,sceneid,objectid,componentid
0,637599863429646302,Ubiq.Messaging.NetworkScene,Awake,37524d0d-01b5cb2f,DESKTOP-F1J0MRR,System Product Name (ASUS),f73fe01b1e21031d49274a1491d1d6b5714c92e9,NaN,NaN,NaN
1,637599863596028409,Ubiq.Voip.VoipPeerConnectionManager,CreatePeerConnectionForPeer,5189a9df-ef0fb2fa,74239db7-e985a872,NaN,NaN,37524d0d-01b5cb2f,37524d0d-01b5cb2f,50.0
2,637599863596093552,Ubiq.Voip.VoipPeerConnectionManager,RequestPeerConnection,5189a9df-ef0fb2fa,74239db7-e985a872,NaN,NaN,37524d0d-01b5cb2f,37524d0d-01b5cb2f,50.0
3,637599863596113552,Ubiq.Samples.NetworkSpawner,SpawnObject,2,80929f8b-563730ba,True,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0
4,637599863596153581,Ubiq.Samples.NetworkSpawner,SpawnObject,2,5bcc6dba-d076d69a,True,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0


We can use Pandas to filter and process the structured logs. Use Unique to find all the event types seen during the session.

In [7]:
df.type.unique()

array(['Ubiq.Messaging.NetworkScene',
       'Ubiq.Voip.VoipPeerConnectionManager',
       'Ubiq.Samples.NetworkSpawner'], dtype=object)

Pandas can perform vector comparisons, and filter DataFrames by row indices. Select all the SpawnObject events.

In [23]:
df[df.event == "SpawnObject"].head(5)

,ticks,type,event,arg1,arg2,arg3,arg4,sceneid,objectid,componentid
3,637599863596113552,Ubiq.Samples.NetworkSpawner,SpawnObject,2,80929f8b-563730ba,True,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0
4,637599863596153581,Ubiq.Samples.NetworkSpawner,SpawnObject,2,5bcc6dba-d076d69a,True,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0
5,637599863596178686,Ubiq.Samples.NetworkSpawner,SpawnObject,2,377bea89-4da20ca0,False,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0
6,637599863596188693,Ubiq.Samples.NetworkSpawner,SpawnObject,2,f46f4050-a5381d60,False,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0
7,637599863596193928,Ubiq.Samples.NetworkSpawner,SpawnObject,2,6b18a44c-922b86e8,False,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0


The Shape member shows the size of the result

In [10]:
df[df.event == "SpawnObject"].shape

(44, 10)

The Pandas [*merge*](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) method is used to perform inner and outer joins to relate different log events. We merge the `Awake` events with the `SpawnObject` events based on the `NetworkScene` Ids (`sceneid` in from `ContextEventLogger` for `SpawnObject`, and `arg1` for `Awake`).

In [31]:
spawn = df[df.event == "SpawnObject"]
awake = df[df.event == "Awake"]
f = pd.merge(spawn,awake,how="left",left_on="sceneid",right_on="arg1")
f.head(5)

,ticks_x,type_x,event_x,arg1_x,arg2_x,arg3_x,arg4_x,sceneid_x,objectid_x,componentid_x,ticks_y,type_y,event_y,arg1_y,arg2_y,arg3_y,arg4_y,sceneid_y,objectid_y,componentid_y
0,637599863596113552,Ubiq.Samples.NetworkSpawner,SpawnObject,2,80929f8b-563730ba,True,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0,637599863429646302,Ubiq.Messaging.NetworkScene,Awake,37524d0d-01b5cb2f,DESKTOP-F1J0MRR,System Product Name (ASUS),f73fe01b1e21031d49274a1491d1d6b5714c92e9,NaN,NaN,NaN
1,637599863596153581,Ubiq.Samples.NetworkSpawner,SpawnObject,2,5bcc6dba-d076d69a,True,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0,637599863429646302,Ubiq.Messaging.NetworkScene,Awake,37524d0d-01b5cb2f,DESKTOP-F1J0MRR,System Product Name (ASUS),f73fe01b1e21031d49274a1491d1d6b5714c92e9,NaN,NaN,NaN
2,637599863596178686,Ubiq.Samples.NetworkSpawner,SpawnObject,2,377bea89-4da20ca0,False,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0,637599863429646302,Ubiq.Messaging.NetworkScene,Awake,37524d0d-01b5cb2f,DESKTOP-F1J0MRR,System Product Name (ASUS),f73fe01b1e21031d49274a1491d1d6b5714c92e9,NaN,NaN,NaN
3,637599863596188693,Ubiq.Samples.NetworkSpawner,SpawnObject,2,f46f4050-a5381d60,False,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0,637599863429646302,Ubiq.Messaging.NetworkScene,Awake,37524d0d-01b5cb2f,DESKTOP-F1J0MRR,System Product Name (ASUS),f73fe01b1e21031d49274a1491d1d6b5714c92e9,NaN,NaN,NaN
4,637599863596193928,Ubiq.Samples.NetworkSpawner,SpawnObject,2,6b18a44c-922b86e8,False,NaN,37524d0d-01b5cb2f,7725a971-a3692643,49018.0,637599863429646302,Ubiq.Messaging.NetworkScene,Awake,37524d0d-01b5cb2f,DESKTOP-F1J0MRR,System Product Name (ASUS),f73fe01b1e21031d49274a1491d1d6b5714c92e9,NaN,NaN,NaN


We can perform arithmetic operations too. We use [string](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html) operations, boolean arrays and [size](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.size.html) to find the number of distinct objects spawned by the Oculus Quest.

In [51]:
is_quest = f.arg2_y.astype(str).str.contains("Quest")
is_owner = f.arg3_x.astype(bool)
spawned_ids = f[is_quest & is_owner].arg2_x
spawned_ids.unique().size


13